In [20]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

import pandas as pd
import os

In [21]:
def FFNN(input_shape,
         n_neurons,
         n_hidden,
         learning_rate):
    
    # neural network dynamic architecture
    model = Sequential()
    model.add(Dense(n_neurons, input_shape=(input_shape,), activation='relu'))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu")) 
    model.add(Dense(1))

    # define stochastic gradient descent optimizer
    optimizer = SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return model

In [22]:
SOURCE_PATH = os.path.dirname(os.getcwd())

N_JOBS = -1  # number of jobs to run in parallel
N_SPLITS = 10  # number of splits (k) to be made within the k fold cv
N_ITER = 50  # number of parameter settings that are sampled
SEED = 2294
VERBOSE = True
INPUTS_PATH = os.path.join(SOURCE_PATH, "data", "inputs")
OUTPUTS_PATH = os.path.join(SOURCE_PATH, "data", "outputs")
DATASET_NAMES = ["betadgp_covdgp_data", "betadgp_beta2x2_data", "betadgp_data"]
TARGET_NAME = "betas_dgp"
MODEL_TAG = "ffnn"
STANDARDIZE = True
TRAIN_SIZE = 0.7 # size of training data (in %)
DIR_NAMES = os.listdir(os.path.join(INPUTS_PATH))

VALIDATION_SIZE = 0.2

In [23]:
# train data
train_data = pd.read_csv(os.path.join(INPUTS_PATH, DIR_NAMES[0], DATASET_NAMES[0] + ".csv"))
train_data.set_index(["Var1", "Var2"], inplace=True)
y_train = train_data[[TARGET_NAME]].to_numpy()
X_train = train_data.drop([TARGET_NAME], axis=1).to_numpy()

# test data
test_data = pd.read_csv(os.path.join(INPUTS_PATH, DIR_NAMES[0], DATASET_NAMES[0] + "_test.csv"))
test_data.set_index(["Var1", "Var2"], inplace=True)
y_test = test_data[[TARGET_NAME]].to_numpy()
X_test = test_data.drop([TARGET_NAME], axis=1).to_numpy()

# standardize if needed
if STANDARDIZE:
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=TRAIN_SIZE)

    scaler = StandardScaler()
    X_train_zscore = scaler.fit_transform(X_train)
    X_validation_zscore = scaler.transform(X_validation)
    X_test_zscore = scaler.transform(X_test)

In [25]:
# wrap-up dynamic FFNN
model = keras.wrappers.scikit_learn.KerasRegressor(FFNN, verbose=0)

# define the grid search parameters
param_grid = dict(
    
    batch_size=[10, 20, 40, 60, 80, 100],
    epochs=[10, 50, 100],
    input_shape=[X_train.shape[1]],
    n_hidden=range(1, 1 + 1),
    n_neurons=[10],
    learning_rate=[0.001, 0.01, 0.1, 0.2, 0.3]
                 
)

# define data splits for the cross validation
cv_splits = KFold(n_splits=N_SPLITS)

# define hyperparameter optimization
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=N_JOBS,
                    cv=cv_splits)

# train model
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.004123 using {'batch_size': 40, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.01, 'n_hidden': 1, 'n_neurons': 10}


In [26]:
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.004261 (0.000497) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.001, 'n_hidden': 1, 'n_neurons': 10}
-0.004158 (0.000511) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.01, 'n_hidden': 1, 'n_neurons': 10}
-0.004173 (0.000470) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.1, 'n_hidden': 1, 'n_neurons': 10}
-0.004427 (0.000901) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.2, 'n_hidden': 1, 'n_neurons': 10}
-0.004295 (0.000595) with: {'batch_size': 10, 'epochs': 10, 'input_shape': 1, 'learning_rate': 0.3, 'n_hidden': 1, 'n_neurons': 10}
-0.004162 (0.000474) with: {'batch_size': 10, 'epochs': 50, 'input_shape': 1, 'learning_rate': 0.001, 'n_hidden': 1, 'n_neurons': 10}
-0.004162 (0.000508) with: {'batch_size': 10, 'epochs': 50, 'input_shape': 1, 'learning_rate': 0.01, 'n_hidden': 1, 'n_neurons': 10}
-0.004204 (0.000498) with: {'batch_size': 10, 'epochs': 50, 'input_sha